Ein Skript zur Extrahierung der Versuchsdaten aus der Datenbank der Edge-Box

In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
from keras.models import Model, load_model
import glob 
import  pandas as pd
import matplotlib.pyplot as plt
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
import os
from pymongo import MongoClient
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta
import plotly.io as pio
from time import sleep

DB Config

In [ ]:
#####DB Config
host= "DevEdgeV32"
port=27017

Set parameters

In [ ]:
save_path= 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\Edgebox\\Versuchsdaten\\5s_data\\'#neue_Spindel\\'
output_variable= 'welle_z'
prediction_variable= 'prediction'
csv_selected_columns= ['date','t_bett','t_motor', 't_spindle','DRZ2','M8', 'M121', 'M127', 'M7','given2model', output_variable, prediction_variable]
all_data= True
get_preprocessed_data= False

Set time priod needed

In [ ]:
# start= datetime.strptime("2022-09-20T22:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")# Trocken iso
# end= datetime.strptime ("2023-02-01T08:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
#start= datetime.strptime("2023-01-20T00:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000") # Trocken Fräsen
#end= datetime.strptime ("2023-01-24T00:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
# start= datetime.strptime("2023-01-23T10:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000") # nass Fräsen
# end= datetime.strptime ("2023-01-23T17:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
start= datetime.strptime("2023-03-01T11:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
end= datetime.strptime ("2023-03-01T17:59:59.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")

Get Data from database

In [ ]:
## Read raw and preprocessing data
client = MongoClient(host= host, port=port)
db = client.h4ai
event_list = db.modelLogs.find({ "date" : { '$gte' : start, '$lt' : end} }).sort('date', 1)
signals= None
i=0
last_prediction= -500
prev_time= None
current_time= None
stop= False
### Go through events in DB
for event in event_list:
    i+=1
    record_list= event['content']
    keys= None
    ### Go through records in each event
    for record in record_list:
        ### Get right data between start / end 
        if record['date']< start or record['date'] > end:
            continue
        current_time= record['date']#datetime.strptime(record['date'], "%Y-%m-%d %H:%M:%S.%f")
        # if prev_time is None:
        #     prev_time= current_time
        # # elif (current_time - prev_time).total_seconds()>10:
        # #     #print('prev_time: {time}'.format(time= prev_time))
        # #     #print(print('current_time: {time}'.format(time= current_time)))
        # #     stop= True
        # #     break
        # # else:
        # #     prev_time= current_time
        # # if stop:
        # #     break
    ##### Get only data points to be predicted
        if not all_data:
            if 'given2model' in record:
                if not record['given2model']:
                    continue
            elif record[prediction_variable] == last_prediction:
                continue
            else:
                last_prediction= record[prediction_variable]
        #####
        if 'given2model' in record:
           record['raw_data']['given2model'] =record['given2model']
        keys= record['raw_data'].keys()
        for item in keys:
            record['raw_data'][item]= [record['raw_data'][item]]
        #print ('before',record)
        record['raw_data']['date']=[record['date'] + timedelta(hours= +1)]
        record['raw_data'][output_variable]=[record[output_variable]]
        record['raw_data'][prediction_variable]=[record[prediction_variable]]
        record['raw_data']['DRZ2']= float(record['DRZ2'])
        #record['raw_data']['DRZ2'][0]= float(record['raw_data']['DRZ2'][0])
        record['raw_data']['t_spindle'][0]= np.round(record['raw_data']['t_spindle'][0], decimals=2)
        record['raw_data']['t_motor']= np.round(float(record['raw_data']['t_motor'][0]), decimals= 2)
        # record['raw_data']['welle_std_comp']=[record['std_comp']]
        # record['raw_data']['schlitten_comp']=[record['schlitten_comp']]
        # record['raw_data']['hals_comp']=[record['hals_comp']]
        # record['raw_data']['drz_comp']=[record['drz_comp']]
        # record['raw_data']['tcp_z']= [record['tcp_z']]
        #####Add preprocessed Data##########
        if get_preprocessed_data:
            record['raw_data']['0']= record['preprocessed_data'][0]
            record['raw_data']['1']= record['preprocessed_data'][1]
            record['raw_data']['2']= record['preprocessed_data'][2]
            record['raw_data']['3']= record['preprocessed_data'][3]
            record['raw_data']['4']= record['preprocessed_data'][4]
            record['raw_data']['5']= record['preprocessed_data'][5]
        ####################################
        #print ('After ',record)
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if signals is None:
            signals= lf_signal_point
        else:
            signals= signals.append(lf_signal_point,ignore_index= True)
signals.reset_index(inplace= True)
####Apply enterpolation###
upsampled_variable= str('interpolated_'+output_variable)
signals[upsampled_variable]= signals[output_variable]
current_welle_z= 1000000
for index, row in signals.iterrows():
    if row[upsampled_variable]!= current_welle_z: ### A New value is available
        current_welle_z= row[upsampled_variable]
    else:
        signals.at[index,upsampled_variable]= np.NaN
signals[upsampled_variable].interpolate(method='linear', order=3, axis= 0,inplace= True)
output=  signals[output_variable]


In [ ]:
signals.columns

Visualization of the predictions and labels and Error borders (Needed from Andreas)

In [ ]:
# scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
# # selected_Columns= ['t_bett','t_motor', 't_spindle' ,'M8', 'M121', 'M127', 'M7', prediction_variable, output_variable, 'db_prediction_abs_error']
# # y_axis_names= selected_Columns
# #print(preds)
# fig= make_subplots(rows=1,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
# ##Verlagerung
# fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[output_variable], name='Gemessene Verlagerung (Welle)', mode= scatter_mode), row= 1, col= 1)
# fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[prediction_variable], name='KI-basierte Verlagerung (Welle)', mode= scatter_mode), row= 1, col= 1)

# ##Restfehler
# fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[output_variable] - 1000 * signals[prediction_variable], name=' Restfehler', mode= scatter_mode),  row= 1 , col= 1)
# ## Draw the tolerence +-5
# fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable],5), name='+5 Obere Grenze', mode= scatter_mode),  row= 1 , col= 1)
# fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable],-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= 1 , col= 1)
# fig.update_yaxes(title_text= 'Verlagerung Micro-Meter', row= 1, col= 1)
# fig.update_layout(height=1200, width=1400, title_text= 'M57002 Machine Data, Versuch am 2023-01-11T20:00:00 ISO 230-3 Trocken')

Correct the delay of the Welle_z

In [ ]:
### Special for data gathered with new app
#signals[output_variable]= signals[output_variable].shift(-1)


In [ ]:
# length= len(signals)
# start= -1
# end= -1
# phases=[]
# for index, row in signals.iterrows():
#     if start<0:
#         start= index
#         continue
#     if row['given2model'] or index == length - 1:
#         end= index-1
#         new_phase= (start, end)
#         phases.append(new_phase)
#         start= index
        
# for phase in phases:
#     start_idx= phase[0]
#     end_idx= phase[1]
#     #print(phase)
#     #print('in= ', start_idx,':', end_idx + 1, ', from= ', end_idx+1)
#     signals['welle_z'][start_idx: end_idx + 1]= signals['welle_z'][end_idx+1]

Correct the shift of z_welle

In [ ]:
#signals[output_variable].mask(signals[output_variable]>10,signals[prediction_variable], inplace=True)

In [ ]:
# shift_value= signals[output_variable][0]
# signals[output_variable] -= shift_value

In [ ]:
# signals['db_prediction_abs_error']= signals[prediction_variable] -signals[output_variable]
# signals['db_prediction_abs_error']= 1000 * signals['db_prediction_abs_error']

In [ ]:
#window= 24
#signals['t_motor']= signals['t_motor'].rolling(window=window).mean()
#signals['t_spindle']= signals['t_spindle'].rolling(window= window).mean()
#signals[prediction_variable]= signals[prediction_variable].rolling(window= window).mean()
#signals= signals[window+1:]

In [ ]:
# scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
# # selected_Columns= ['t_bett','t_motor', 't_spindle' ,'M8', 'M121', 'M127', 'M7', prediction_variable, output_variable, 'db_prediction_abs_error']
# # y_axis_names= selected_Columns
# #print(preds)
# fig= make_subplots(rows=8,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
# ##DRZ2
# fig.add_trace(go.Scatter(x= signals['date'], y= signals['DRZ2'], name='DRZ2', mode= scatter_mode), row= 1, col= 1)
# fig.update_yaxes(title_text= 'DRZ (RPM)', row= 1, col= 1)
# ##t_motor
# fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_motor'], name='t_motor', mode= scatter_mode), row= 2, col= 1)
# fig.update_yaxes(title_text= 't_motor C°', row= 2, col= 1)
# ##t_spindel
# fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_spindle'], name='t_spindel', mode= scatter_mode), row= 3, col= 1)
# fig.update_yaxes(title_text= 't_spindel C°', row= 3, col= 1)
# ##'hals'
# fig.add_trace(go.Scatter(x= signals['date'], y= -1000*signals['hals_comp'], name='hals_comp', mode= scatter_mode), row= 4, col= 1)
# fig.update_yaxes(title_text= 'hals_comp MM', row= 4, col= 1)
# ##'Schlitten'
# fig.add_trace(go.Scatter(x= signals['date'], y= -1000*signals['schlitten_comp'], name='schlitten_comp', mode= scatter_mode), row= 5, col= 1)
# fig.update_yaxes(title_text= 'schlitten_comp MM', row= 5, col= 1)
# ##Verlagerung
# # fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals['compensation'], name='standard Verlagerung (Welle)', mode= scatter_mode), row= 9, col= 1)
# fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[prediction_variable], name='KI-basierte Verlagerung (Welle)', mode= scatter_mode), row= 6, col= 1)
# fig.update_yaxes(title_text= 'Verlagerung MM', row= 6, col= 1)
# ##drz_comp
# fig.add_trace(go.Scatter(x= signals['date'], y= - 1000 * signals['drz_comp'], name='drz_comp', mode= scatter_mode), row= 7, col= 1)
# fig.update_yaxes(title_text= 'Verlagerung DRZ MM', row= 7, col= 1)
# ##tcp_z
# fig.add_trace(go.Scatter(x= signals['date'], y= -1000 * (signals['drz_comp']+ signals['hals_comp'] +signals['schlitten_comp']-signals[prediction_variable]), name='KI_Verlagerung', mode= scatter_mode), row= 8, col= 1)
# fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals['tcp_z'], name='tcp_z Verlagerung', mode= scatter_mode), row= 8, col= 1)
# fig.update_yaxes(title_text= 'tcp_z Verlagerung MM', row= 8, col= 1)
# fig.update_layout(height=1200, width=1400, title_text= 'M57002 Machine Data')
#fig.show()

In [ ]:
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
# selected_Columns= ['t_bett','t_motor', 't_spindle' ,'M8', 'M121', 'M127', 'M7', prediction_variable, output_variable, 'db_prediction_abs_error']
# y_axis_names= selected_Columns
#print(preds)
fig= make_subplots(rows=10,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
#DRZ2
fig.add_trace(go.Scatter(x= signals['date'], y= signals['DRZ2'], name='DRZ2', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 'DRZ (RPM)', row= 1, col= 1)
##t_bett
fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_bett'], name='t_bett', mode= scatter_mode), row= 2, col= 1)
fig.update_yaxes(title_text= 't_bett C°', row= 2, col= 1)
##t_motor
fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_motor'], name='t_motor', mode= scatter_mode), row= 3, col= 1)
fig.update_yaxes(title_text= 't_motor C°', row= 3, col= 1)
##t_spindel
fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_spindle'], name='t_spindel', mode= scatter_mode), row= 4, col= 1)
fig.update_yaxes(title_text= 't_spindel C°', row= 4, col= 1)
##'M8'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M8'], name='M8', mode= scatter_mode), row= 5, col= 1)
fig.update_yaxes(title_text= 'M8 0/1', row= 5, col= 1)
##'M121'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M121'], name='M121', mode= scatter_mode), row= 6, col= 1)
fig.update_yaxes(title_text= 'M121 0/1', row= 6, col= 1)
##'M127'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M127'], name='M127', mode= scatter_mode), row= 7, col= 1)
fig.update_yaxes(title_text= 'M127 0/1', row= 7, col= 1)
##'M7'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M7'], name='M7', mode= scatter_mode), row= 8, col= 1)
fig.update_yaxes(title_text= 'M7 0/1', row= 8, col= 1)
##Verlagerung
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[upsampled_variable], name='interpolierte Messtasterwerte (Welle)', mode= scatter_mode), row= 9, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[output_variable], name='Messtasterwerte (Welle)', mode= scatter_mode), row= 9, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[prediction_variable], name='KI-basierte Verlagerung (Welle)', mode= scatter_mode), row= 9, col= 1)
fig.update_yaxes(title_text= 'Verlagerung Micro-Meter', row= 9, col= 1)
##Restfehler
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[upsampled_variable] - 1000 * signals[prediction_variable], name=' Restfehler', mode= scatter_mode),  row= 10 , col= 1)
## Draw the tolerence +-5
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable],5), name='+5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable],-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
fig.update_yaxes(title_text= output_variable, row= 10, col= 1)
fig.update_layout(height=1200, width=1400, title_text= 'M57002 Machine Data')
fig.show()

In [ ]:
signals.to_csv('Fraesversuchtrocken_iso_01032023.csv')

Static Visualization

In [ ]:
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
# selected_Columns= ['t_bett','t_motor', 't_spindle' ,'M8', 'M121', 'M127', 'M7', prediction_variable, output_variable, 'db_prediction_abs_error']
# y_axis_names= selected_Columns
#print(preds)
fig= make_subplots(rows=10,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
##DRZ2
#fig.add_trace(go.Scatter(x= signals['date'], y= signals['DRZ2'], name='DRZ2', mode= scatter_mode), row= 1, col= 1)
#fig.update_yaxes(title_text= 'DRZ (RPM)', row= 1, col= 1)
##t_bett
fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_bett'], name='t_bett', mode= scatter_mode), row= 2, col= 1)
fig.update_yaxes(title_text= 't_bett C°', row= 2, col= 1)
##t_motor
fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_motor'], name='t_motor', mode= scatter_mode), row= 3, col= 1)
fig.update_yaxes(title_text= 't_motor C°', row= 3, col= 1)
##t_spindel
fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_spindle'], name='t_spindel', mode= scatter_mode), row= 4, col= 1)
fig.update_yaxes(title_text= 't_spindel C°', row= 4, col= 1)
##'M8'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M8'], name='M8', mode= scatter_mode), row= 5, col= 1)
fig.update_yaxes(title_text= 'M8 0/1', row= 5, col= 1)
##'M121'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M121'], name='M121', mode= scatter_mode), row= 6, col= 1)
fig.update_yaxes(title_text= 'M121 0/1', row= 6, col= 1)
##'M127'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M127'], name='M127', mode= scatter_mode), row= 7, col= 1)
fig.update_yaxes(title_text= 'M127 0/1', row= 7, col= 1)
##'M7'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M7'], name='M7', mode= scatter_mode), row= 8, col= 1)
fig.update_yaxes(title_text= 'M7 0/1', row= 8, col= 1)
##Verlagerung
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[upsampled_variable], name='interpolierte Messtasterwerte (Welle)', mode= scatter_mode), row= 9, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[output_variable], name='Messtasterwerte (Welle)', mode= scatter_mode), row= 9, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[prediction_variable], name='KI-basierte Verlagerung (Welle)', mode= scatter_mode), row= 9, col= 1)
fig.update_yaxes(title_text= 'Verlagerung Micro-Meter', row= 9, col= 1)
##Restfehler
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[upsampled_variable] - 1000 * signals[prediction_variable], name=' Restfehler', mode= scatter_mode),  row= 10 , col= 1)
## Draw the tolerence +-5
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable],5), name='+5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable],-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
fig.update_yaxes(title_text= output_variable, row= 10, col= 1)
fig.update_layout(height=1200, width=1400, title_text= 'M57002 Machine Data')
fig.show()

In [ ]:
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
# selected_Columns= ['t_bett','t_motor', 't_spindle' ,'M8', 'M121', 'M127', 'M7', prediction_variable, output_variable, 'db_prediction_abs_error']
# y_axis_names= selected_Columns
#print(preds)
fig= make_subplots(rows=10,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
##DRZ2
#fig.add_trace(go.Scatter(x= signals['date'], y= signals['DRZ2'], name='DRZ2', mode= scatter_mode), row= 1, col= 1)
#fig.update_yaxes(title_text= 'DRZ (RPM)', row= 1, col= 1)
##t_bett
fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_bett'], name='t_bett', mode= scatter_mode), row= 2, col= 1)
fig.update_yaxes(title_text= 't_bett C°', row= 2, col= 1)
##t_motor
fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_motor'], name='t_motor', mode= scatter_mode), row= 3, col= 1)
fig.update_yaxes(title_text= 't_motor C°', row= 3, col= 1)
##t_spindel
fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_spindle'], name='t_spindel', mode= scatter_mode), row= 4, col= 1)
fig.update_yaxes(title_text= 't_spindel C°', row= 4, col= 1)
##'M8'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M8'], name='M8', mode= scatter_mode), row= 5, col= 1)
fig.update_yaxes(title_text= 'M8 0/1', row= 5, col= 1)
##'M121'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M121'], name='M121', mode= scatter_mode), row= 6, col= 1)
fig.update_yaxes(title_text= 'M121 0/1', row= 6, col= 1)
##'M127'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M127'], name='M127', mode= scatter_mode), row= 7, col= 1)
fig.update_yaxes(title_text= 'M127 0/1', row= 7, col= 1)
##'M7'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M7'], name='M7', mode= scatter_mode), row= 8, col= 1)
fig.update_yaxes(title_text= 'M7 0/1', row= 8, col= 1)
##Verlagerung
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[upsampled_variable], name='interpolierte Messtasterwerte (Welle)', mode= scatter_mode), row= 9, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[output_variable], name='Messtasterwerte (Welle)', mode= scatter_mode), row= 9, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[prediction_variable], name='KI-basierte Verlagerung (Welle)', mode= scatter_mode), row= 9, col= 1)
fig.update_yaxes(title_text= 'Verlagerung Micro-Meter', row= 9, col= 1)
##Restfehler
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[upsampled_variable] - 1000 * signals[prediction_variable], name=' Restfehler', mode= scatter_mode),  row= 10 , col= 1)
## Draw the tolerence +-5
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable],5), name='+5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable],-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
fig.update_yaxes(title_text= output_variable, row= 10, col= 1)
fig.update_layout(height=1200, width=1400, title_text= 'M57002 Machine Data')
fig.show()

Interactive visualization

In [ ]:
# scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
# fig_subplot= make_subplots(rows=10,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
# fig= go.FigureWidget(fig_subplot)l
# fig.update_yaxes(title_text= 'DRZ (RPM)', row= 1, col= 1)
# ##t_bett
# fig.update_yaxes(title_text= 't_bett C°', row= 2, col= 1)
# ##t_motor
# fig.update_yaxes(title_text= 't_motor C°', row= 3, col= 1)
# ##t_spindel
# fig.update_yaxes(title_text= 't_spindel C°', row= 4, col= 1)
# ##'M8'
# fig.update_yaxes(title_text= 'M8 0/1', row= 5, col= 1)
# ##'M121'
# fig.update_yaxes(title_text= 'M121 0/1', row= 6, col= 1)
# ##'M127'
# fig.update_yaxes(title_text= 'M127 0/1', row= 7, col= 1)
# ##'M7'
# fig.update_yaxes(title_text= 'M7 0/1', row= 8, col= 1)
# ##Verlagerung
# fig.update_yaxes(title_text= 'Verlagerung Micro-Meter', row= 9, col= 1)
# ##Restfehler
# fig.update_yaxes(title_text= output_variable, row= 10, col= 1)
# fig.update_layout(height=800, width=1400, title_text= 'M57002 Machine Data')


In [ ]:
# for i in range(len(signals)):
#     if i == 0:
#         ##DRZ2
#         fig.add_trace(go.Scatter(x= signals['date'][0:i+1], y= signals['DRZ2'][0:i+1], name='DRZ2', mode= scatter_mode), row= 1, col= 1)
#         ##t_bett
#         fig.add_trace(go.Scatter(x= signals['date'][0:i+1], y= signals['t_bett'][0:i+1], name='t_bett', mode= scatter_mode), row= 2, col= 1)
#         ##t_motor
#         fig.add_trace(go.Scatter(x= signals['date'][0:i+1], y= signals['t_motor'][0:i+1], name='t_motor', mode= scatter_mode), row= 3, col= 1)
#         ##t_spindel
#         fig.add_trace(go.Scatter(x= signals['date'][0:i+1], y= signals['t_spindle'][0:i+1], name='t_spindel', mode= scatter_mode), row= 4, col= 1)
#         ##'M8'
#         fig.add_trace(go.Scatter(x= signals['date'][0:i+1], y= signals['M8'][0:i+1], name='M8', mode= scatter_mode), row= 5, col= 1)
#         ##'M121'
#         fig.add_trace(go.Scatter(x= signals['date'][0:i+1], y= signals['M121'][0:i+1], name='M121', mode= scatter_mode), row= 6, col= 1)
#         ##'M127'
#         fig.add_trace(go.Scatter(x= signals['date'][0:i+1], y= signals['M127'][0:i+1], name='M127', mode= scatter_mode), row= 7, col= 1)
#         ##'M7'
#         fig.add_trace(go.Scatter(x= signals['date'][0:i+1], y= signals['M7'][0:i+1], name='M7', mode= scatter_mode), row= 8, col= 1)
#         ##Verlagerung
#         # fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals['compensation'], name='standard Verlagerung (Welle)', mode= scatter_mode), row= 9, col= 1)
#         fig.add_trace(go.Scatter(x= signals['date'][0:i+1], y= 1000 * signals[prediction_variable][0:i+1], name='KI-basierte Verlagerung (Welle)', mode= scatter_mode), row= 9, col= 1)
#         ##Restfehler
#         fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[output_variable][0:i+1] - 1000 * signals[prediction_variable][0:i+1], name=' Restfehler', mode= scatter_mode),  row= 10 , col= 1)
#         ## Draw the tolerence +-5
#         fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable][0:i+1],5), name='+5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
#         fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable][0:i+1],-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
#     else:
#         fig.data[0].x=signals['date'][0:i+1]
#         fig.data[0].y=signals['DRZ2'][0:i+1]
#         ##t_bett
#         fig.data[1].x=signals['date'][0:i+1]
#         fig.data[1].y=signals['t_bett'][0:i+1]
#         ##t_motor
#         fig.data[2].x=signals['date'][0:i+1]
#         fig.data[2].y=signals['t_motor'][0:i+1]
#         ##t_spindel
#         #fig.add_trace(go.Scatter(x= signals['date'][0:i+1], y= signals['t_spindle'][0:i+1], name='t_spindel', mode= scatter_mode), row= 4, col= 1)
#         fig.data[3].x=signals['date'][0:i+1]
#         fig.data[3].y=signals['t_spindle'][0:i+1]
#         ##'M8'
#         #fig.add_trace(go.Scatter(x= signals['date'][0:i+1], y= signals['M8'][0:i+1], name='M8', mode= scatter_mode), row= 5, col= 1)
#         fig.data[4].x=signals['date'][0:i+1]
#         fig.data[4].y=signals['M8'][0:i+1]
#         ##'M121'
#         #fig.add_trace(go.Scatter(x= signals['date'][0:i+1], y= signals['M121'][0:i+1], name='M121', mode= scatter_mode), row= 6, col= 1)
#         fig.data[5].x=signals['date'][0:i+1]
#         fig.data[5].y=signals['M121'][0:i+1]
#         ##'M127'
#         #fig.add_trace(go.Scatter(x= signals['date'][0:i+1], y= signals['M127'][0:i+1], name='M127', mode= scatter_mode), row= 7, col= 1)
#         fig.data[6].x=signals['date'][0:i+1]
#         fig.data[6].y=signals['M127'][0:i+1]
#         ##'M7'
#         #fig.add_trace(go.Scatter(x= signals['date'][0:i+1], y= signals['M7'][0:i+1], name='M7', mode= scatter_mode), row= 8, col= 1)
#         fig.data[7].x=signals['date'][0:i+1]
#         fig.data[7].y=signals['M7'][0:i+1]
#         ##Verlagerung
#         # fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals['compensation'], name='standard Verlagerung (Welle)', mode= scatter_mode), row= 9, col= 1)
#         #fig.add_trace(go.Scatter(x= signals['date'][0:i+1], y= 1000 * signals[prediction_variable][0:i+1], name='KI-basierte Verlagerung (Welle)', mode= scatter_mode), row= 9, col= 1)
#         fig.data[8].x=signals['date'][0:i+1]
#         fig.data[8].y= 1000 * signals[prediction_variable][0:i+1]
#         ##Restfehler
#         #fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[output_variable][0:i+1] - 1000 * signals[prediction_variable][0:i+1], name=' Restfehler', mode= scatter_mode),  row= 10 , col= 1)
#         fig.data[9].x=signals['date'][0:i+1]
#         fig.data[9].y=1000 * signals[output_variable][0:i+1]
#         ## Draw the tolerence +-5
#         #fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable][0:i+1],5), name='+5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
#         fig.data[10].x=signals['date'][0:i+1]
#         fig.data[10].y= np.full_like(signals[output_variable][0:i+1],5)
#         #fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable][0:i+1],-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
#         fig.data[11].x=signals['date'][0:i+1]
#         fig.data[11].y=np.full_like(signals[output_variable][0:i+1],-5)
#     sleep(0.2)

Save data to .csv file

In [ ]:
# ###Save to .CSV-File######
# csv_df= signals[csv_selected_columns]
# csv_df[output_variable]= csv_df[output_variable].round(6)
# csv_df[csv_selected_columns].to_csv(str(save_path+'Versuch20_01_2023_trocken_Fraesen.csv'))